In [1]:
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

### Step 1: Define a molecule
Using the sto3g basis with the PySCF driver as an example, we can describe a molecule.

In [2]:
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

### Step 2: Prepare qubit Hamiltonian
Here, we setup the molecular orbitals to be considered and can reduce the problem size when we map to the qubit Hamiltonian. 

We are also able to define the mapping type for the qubit Hamiltonian, which - in some cases - allows one to further reduce the problem size.

In [3]:
map_type = 'parity'

In [4]:
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order

In [5]:
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -8.854072040283647
# of electrons: 4
# of spin orbitals: 12


In [6]:
# prepare full idx of freeze_list and remove_list
# convert all negative idx to positive
remove_list = [x % molecule.num_orbitals for x in remove_list]
freeze_list = [x % molecule.num_orbitals for x in freeze_list]
# update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
remove_list = [x - len(freeze_list) for x in remove_list]
remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
freeze_list += [x + molecule.num_orbitals for x in freeze_list]

# prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
# and if PARITY mapping is selected, reduction qubits
energy_shift = 0.0
qubit_reduction = True if map_type == 'parity' else False

ferOp = FermionicOperator(h1=h1, h2=h2)
if len(freeze_list) > 0:
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
if len(remove_list) > 0:
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)

qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
qubitOp.chop(10**-10)

print(qubitOp)
print(qubitOp.print_details())

Representation: paulis, qubits: 4, size: 100
IIII	(-0.2076593350197093+0j)
IIIZ	(-0.09376337484626251+0j)
IIZX	(-0.0031775814548396244+0j)
IIIX	(0.0031775814548396244+0j)
IIXX	(-0.0012513965999699584+0j)
IIYY	(0.0012513965999699584+0j)
IIZZ	(-0.21162509515110095+0j)
IIXZ	(0.01920053386310993+0j)
IIXI	(0.01920053386310993+0j)
IIZI	(0.358102699457702+0j)
IZII	(0.09376337484626249+0j)
ZXII	(0.003177581454839621+0j)
IXII	(0.003177581454839621+0j)
XXII	(-0.0012513965999699714+0j)
YYII	(0.0012513965999699714+0j)
ZZII	(-0.21162509515110095+0j)
XZII	(-0.019200533863109923+0j)
XIII	(0.019200533863109923+0j)
ZIII	(-0.358102699457702+0j)
IZIZ	(-0.12182774215821472+0j)
IZZX	(0.012144897228064262+0j)
IZIX	(-0.012144897228064262+0j)
IZXX	(0.03169874598734438+0j)
IZYY	(-0.03169874598734438+0j)
IXIZ	(0.012144897228064262+0j)
ZXIZ	(0.012144897228064262+0j)
IXZX	(-0.003265995499658126+0j)
ZXZX	(-0.003265995499658126+0j)
IXIX	(0.003265995499658126+0j)
ZXIX	(0.003265995499658126+0j)
IXXX	(-0.0086501568606